<a href="https://colab.research.google.com/github/Harris-Gacheru/Vehicle-body-type-classifier/blob/main/Vehicle_Body_Type_Classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"harrisgacheru","key":"084769ba7bb7dc95504e23c2d736700f"}'}

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download -d ademboukhris/cars-body-type-cropped
!unzip -q cars-body-type-cropped.zip -d dataset

# Restructure dataset
!mv dataset/Cars_Body_Type/* dataset/
!rm -rf dataset/Cars_Body_Type


Dataset URL: https://www.kaggle.com/datasets/ademboukhris/cars-body-type-cropped
License(s): CC0-1.0
100% 1.33G/1.34G [00:06<00:00, 255MB/s]
100% 1.34G/1.34G [00:06<00:00, 224MB/s]


In [ ]:
import os

DATA_DIR = '/content/dataset'
TRAIN_DIR =  os.path.join(DATA_DIR, 'train')
VALID_DIR = os.path.join(DATA_DIR, 'valid')
TEST_DIR  = os.path.join(DATA_DIR, 'test')
IMAGE_SIZE = (299, 299)
BATCH_SIZE = 16
SEED = 42
EPOCHS_HEAD = 15
EPOCHS_FINE = 30
FINE_TUNE_AT = -30
MODEL_SAVE_PATH = '/content/model/efficientnet_bodytype_best.h5'

In [ ]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20,
    width_shift_range=0.12,
    height_shift_range=0.12,
    shear_range=0.08,
    zoom_range=0.15,
    horizontal_flip=True,
    brightness_range=[0.8,1.2],
    fill_mode='constant'
)

train_generator = train_datagen.flow_from_directory(
    TRAIN_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED,
    interpolation='lanczos'
)

val_datagen = ImageDataGenerator(rescale=1./255)
validation_generator = val_datagen.flow_from_directory(
    VALID_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    seed=SEED,
    interpolation='lanczos'
)

test_datagen = ImageDataGenerator(rescale=1./255)
test_generator = test_datagen.flow_from_directory(
    TEST_DIR,
    target_size=IMAGE_SIZE,
    batch_size=BATCH_SIZE,
    class_mode='categorical',
    shuffle=False,
    interpolation='lanczos'
)

# Print number of images per generator
print("Train samples:", train_generator.samples)
print("Val samples:", validation_generator.samples)
print("Test samples:", test_generator.samples)

# Check class mapping and counts ---
print('Train classes:', train_generator.class_indices)
print('Validation classes:', validation_generator.class_indices)
print('Num classes (train):', train_generator.num_classes, ' Num classes (val):', validation_generator.num_classes)

Found 5350 images belonging to 7 classes.
Found 1397 images belonging to 7 classes.
Found 802 images belonging to 7 classes.
Train samples: 5350
Val samples: 1397
Test samples: 802
Train classes: {'Convertible': 0, 'Coupe': 1, 'Hatchback': 2, 'Pick-Up': 3, 'SUV': 4, 'Sedan': 5, 'VAN': 6}
Validation classes: {'Convertible': 0, 'Coupe': 1, 'Hatchback': 2, 'Pick-Up': 3, 'SUV': 4, 'Sedan': 5, 'VAN': 6}
Num classes (train): 7  Num classes (val): 7


In [ ]:
# New model proposed
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout
from tensorflow.keras.models import Model

base = EfficientNetB0(include_top=False, weights='imagenet', input_shape=(IMAGE_SIZE[0],IMAGE_SIZE[1],3))
base.trainable = False

x = GlobalAveragePooling2D()(base.output)
x = Dropout(0.3)(x)
num_classes = train_generator.num_classes
output = Dense(num_classes, activation='softmax')(x)

model = Model(base.input, output)
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()


16705208/16705208 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer         │ (None, 299, 299,  │          0 │ -                 │
│ (InputLayer)        │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling           │ (None, 299, 299,  │          0 │ input_layer[0][0] │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ normalization       │ (None, 299, 299,  │          7 │ rescaling[0][0]   │
│ (Normalization)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ rescaling_1         │ (None, 299, 299,  │          0 │ normalization[0]… │
│ (Rescaling)         │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv_pad       │ (None, 301, 301,  │          0 │ rescaling_1[0][0] │
│ (ZeroPadding2D)     │ 3)                │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_conv (Conv2D)  │ (None, 150, 150,  │        864 │ stem_conv_pad[0]… │
│                     │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_bn             │ (None, 150, 150,  │        128 │ stem_conv[0][0]   │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ stem_activation     │ (None, 150, 150,  │          0 │ stem_bn[0][0]     │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_dwconv      │ (None, 150, 150,  │        288 │ stem_activation[… │
│ (DepthwiseConv2D)   │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_bn          │ (None, 150, 150,  │        128 │ block1a_dwconv[0… │
│ (BatchNormalizatio… │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_activation  │ (None, 150, 150,  │          0 │ block1a_bn[0][0]  │
│ (Activation)        │ 32)               │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_squeeze  │ (None, 32)        │          0 │ block1a_activati… │
│ (GlobalAveragePool… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reshape  │ (None, 1, 1, 32)  │          0 │ block1a_se_squee… │
│ (Reshape)           │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_reduce   │ (None, 1, 1, 8)   │        264 │ block1a_se_resha… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_expand   │ (None, 1, 1, 32)  │        288 │ block1a_se_reduc… │
│ (Conv2D)            │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_se_excite   │ (None, 150, 150,  │          0 │ block1a_activati… │
│ (Multiply)          │ 32)               │            │ block1a_se_expan… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ block1a_project_co… │ (None, 150, 150,  │        512 │ block1a_se_excit

 Total params: 4,058,538 (15.48 MB)

 Trainable params: 8,967 (35.03 KB)

 Non-trainable params: 4,049,571 (15.45 MB)

In [ ]:
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau, ModelCheckpoint

# class weights
class_weights = compute_class_weight(
    class_weight='balanced',
    classes=np.unique(train_generator.classes),
    y=train_generator.classes
)
class_weights = dict(enumerate(class_weights))

# callbacks
callbacks = [
    EarlyStopping(monitor='val_loss', patience=7, restore_best_weights=True, verbose=1),
    ReduceLROnPlateau(monitor='val_loss', factor=0.3, patience=3, min_lr=1e-6, verbose=1),
    ModelCheckpoint(MODEL_SAVE_PATH, monitor='val_accuracy', save_best_only=True, verbose=1)
]

In [ ]:
history = model.fit(
    train_generator,
    epochs=EPOCHS_HEAD,
    validation_data=validation_generator,
    class_weight=class_weights,
    callbacks=callbacks,
)

/usr/local/lib/python3.12/dist-packages/keras/src/trainers/data_adapters/py_dataset_adapter.py:121: UserWarning: Your `PyDataset` class should call `super().__init__(**kwargs)` in its constructor. `**kwargs` can include `workers`, `use_multiprocessing`, `max_queue_size`. Do not pass these arguments to `fit()`, as they will be ignored.
  self._warn_if_super_not_called()


Epoch 1/15
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1252 - loss: 2.1408
Epoch 1: val_accuracy improved from -inf to 0.11596, saving model to /content/model/efficientnet_bodytype_best.h5


335/335 ━━━━━━━━━━━━━━━━━━━━ 1129s 3s/step - accuracy: 0.1252 - loss: 2.1407 - val_accuracy: 0.1160 - val_loss: 2.1645 - learning_rate: 0.0010
Epoch 2/15
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1375 - loss: 2.1164
Epoch 2: val_accuracy improved from 0.11596 to 0.12169, saving model to /content/model/efficientnet_bodytype_best.h5


335/335 ━━━━━━━━━━━━━━━━━━━━ 1088s 3s/step - accuracy: 0.1375 - loss: 2.1164 - val_accuracy: 0.1217 - val_loss: 2.0618 - learning_rate: 0.0010
Epoch 3/15
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1407 - loss: 2.1001
Epoch 3: val_accuracy improved from 0.12169 to 0.15533, saving model to /content/model/efficientnet_bodytype_best.h5


335/335 ━━━━━━━━━━━━━━━━━━━━ 1120s 3s/step - accuracy: 0.1407 - loss: 2.1001 - val_accuracy: 0.1553 - val_loss: 1.9161 - learning_rate: 0.0010
Epoch 4/15
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1322 - loss: 2.0980
Epoch 4: val_accuracy did not improve from 0.15533
335/335 ━━━━━━━━━━━━━━━━━━━━ 1101s 3s/step - accuracy: 0.1322 - loss: 2.0980 - val_accuracy: 0.0759 - val_loss: 2.1842 - learning_rate: 0.0010
Epoch 5/15
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1190 - loss: 2.0429
Epoch 5: val_accuracy did not improve from 0.15533
335/335 ━━━━━━━━━━━━━━━━━━━━ 1124s 3s/step - accuracy: 0.1191 - loss: 2.0430 - val_accuracy: 0.0759 - val_loss: 2.0942 - learning_rate: 0.0010
Epoch 6/15
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1055 - loss: 2.0924
Epoch 6: ReduceLROnPlateau reducing learning rate to 0.0003000000142492354.

Epoch 6: val_accuracy did not improve from 0.15533
335/335 ━━━━━━━━━━━━━━━━━━━━ 1177s 4s/step - accuracy: 0.1056 - loss: 2.0924 - val_acc

335/335 ━━━━━━━━━━━━━━━━━━━━ 1116s 3s/step - accuracy: 0.1296 - loss: 1.9783 - val_accuracy: 0.2090 - val_loss: 1.9573 - learning_rate: 3.0000e-04
Epoch 10/15
335/335 ━━━━━━━━━━━━━━━━━━━━ 0s 3s/step - accuracy: 0.1191 - loss: 1.9886
Epoch 10: val_accuracy did not improve from 0.20902
335/335 ━━━━━━━━━━━━━━━━━━━━ 1149s 3s/step - accuracy: 0.1191 - loss: 1.9886 - val_accuracy: 0.0759 - val_loss: 1.9492 - learning_rate: 9.0000e-05
Epoch 10: early stopping
Restoring model weights from the end of the best epoch: 3.


In [ ]:
# Evaluation
# Evaluate on validation set
print("\nEvaluating on VALIDATION set...")
val_loss, val_acc = model.evaluate(validation_generator)
print(f'Validation loss: {val_loss:.4f}, Validation accuracy: {val_acc:.4f}')

# Evaluate on test set
print("\nEvaluating on TEST set...")
test_loss, test_acc = model.evaluate(test_generator)
print(f'Test loss: {test_loss:.4f}, Test accuracy: {test_acc:.4f}')


Evaluating on VALIDATION set...
88/88 ━━━━━━━━━━━━━━━━━━━━ 201s 2s/step - accuracy: 0.1136 - loss: 1.8320
Validation loss: 1.9161, Validation accuracy: 0.1553

Evaluating on TEST set...
51/51 ━━━━━━━━━━━━━━━━━━━━ 126s 2s/step - accuracy: 0.0963 - loss: 1.8609
Test loss: 1.9530, Test accuracy: 0.1372


In [ ]:
from tensorflow.keras.preprocessing import image

CLASSES = list(train_generator.class_indices.keys())
print("Class order:", CLASSES)

def predict_vehicle(img_path):
    img = image.load_img(img_path, target_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    prediction = model.predict(img_array)
    class_idx = np.argmax(prediction[0])
    confidence = prediction[0][class_idx]

    print(f"Predicted: {CLASSES[class_idx]}  ({confidence*100:.2f}%)")
    return CLASSES[class_idx], confidence

def predict_vehicle_detailed(img_path):
    """Enhanced prediction with top-5 results"""
    img = image.load_img(img_path, target_size=(IMAGE_SIZE[0], IMAGE_SIZE[1]))
    img_array = image.img_to_array(img) / 255.0
    img_array = np.expand_dims(img_array, axis=0)

    predictions = model.predict(img_array)[0]

    # Get top 5 predictions
    top_indices = np.argsort(predictions)[::-1][:5]

    print(f"\nPredicting: {img_path}")
    print("-" * 50)
    for i, idx in enumerate(top_indices, 1):
        print(f"{i}. {CLASSES[idx]:<15} {predictions[idx]*100:>6.2f}%")

    return CLASSES[top_indices[0]], predictions[top_indices[0]]

Class order: ['Convertible', 'Coupe', 'Hatchback', 'Pick-Up', 'SUV', 'Sedan', 'VAN']


In [ ]:
predict_vehicle_detailed("/content/images/lexus.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 4s 4s/step

Predicting: /content/images/lexus.jpg
--------------------------------------------------
1. Pick-Up          19.30%
2. Convertible      18.29%
3. SUV              18.28%
4. Hatchback        15.30%
5. VAN              12.75%


('Pick-Up', np.float32(0.19300708))

In [ ]:
predict_vehicle_detailed("/content/images/mercedes.jpg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 153ms/step

Predicting: /content/images/mercedes.jpg
--------------------------------------------------
1. Pick-Up          19.41%
2. Convertible      18.27%
3. SUV              18.22%
4. Hatchback        15.16%
5. VAN              12.88%


('Pick-Up', np.float32(0.19409673))

In [ ]:
predict_vehicle_detailed("/content/images/pickup.jpeg")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 151ms/step

Predicting: /content/images/pickup.jpeg
--------------------------------------------------
1. Pick-Up          19.29%
2. SUV              18.36%
3. Convertible      18.32%
4. Hatchback        15.42%
5. VAN              12.56%


('Pick-Up', np.float32(0.19293168))